In [ ]:
import psycopg2 as ps
import ipyleaflet as ipyl
import ipywidgets as ipyw
import pandas as pd
import json


In [ ]:
######
# Preliminary works:
# To extract OpenStreetMap data that we want to display here, 
#  we created filter following transnet project as below:
#  1. All power nodes/ways of relations tagged with power=*"
#  2. All relations tagged with route=power"
#  3. All ways and its corresponding nodes tagged with power=*"
#
# Afterwards, we merge all extracted data before converts it using osm2pgsql 
#  to postGIS-enabled PostgreSQL databases.
######

In [ ]:
######
# This section handle database connection to our postgreSQL database.
# Since ipyleaflet working with GeoJSON format, 
#  we use query database that convert our extracted database to JSON format.
######

# Database, user, and password to be adapted.
conn_string = "dbname='gisosm' user='postgres' password='' host='localhost'"
conn = ps.connect(conn_string)
cur = conn.cursor()
query = """SELECT 
               * 
           FROM 
               planet_osm_line 

"""
columns = """
        operator,
        name,
        highway,
        amenity,
        public_transport,
        railway,
        power,
        route
"""
query_json_line = """
SELECT row_to_json(fc)
 FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
 FROM (SELECT 'Feature' As type
    , ST_AsGeoJSON(ST_Transform(lg.way, 4326))::json As geometry
    , row_to_json((SELECT l FROM (SELECT {}) As l
      )) As properties
   FROM planet_osm_line As lg ) As f )  As fc;
""".format(columns)

######
# Since the big number of points extracted, 
#  we limit the number to 100 randomly selected points 
#  in objective to get faster response of codes when display the map with object plots.
######
query_json_point = """
SELECT row_to_json(fc)
 FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
 FROM (SELECT 'Feature' As type
    , ST_AsGeoJSON(ST_Transform(lg.way, 4326))::json As geometry
    , row_to_json((SELECT l FROM (SELECT {}) As l
      )) As properties
   FROM planet_osm_point As lg order by random() limit 100) As f )  As fc;
""".format(columns)

cur.execute(query_json_point)
d_points = cur.fetchone()
cur.execute(query_json_line)
d_lines = cur.fetchone()

#df = pd.read_sql(query, conn)
conn.close()

In [ ]:
# Loading map area in Berlin.
map = ipyl.Map(center=[52.52, 13.45], zoom=11, layout=ipyw.Layout(height='600px'))
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

# Loading GeoJSON to display extracted data on the map.
layer_points = ipyl.GeoJSON(data=d_points[0], hover_style={'fill_color': 'red'}, style={'fill_color':'#001010','fill':True})
layer_lines = ipyl.GeoJSON(data=d_lines[0],style={'color':'red'})

######
# Function hover_handler handles hover action to show label properties below the map. 
# But we found that ipyleaflet 'on_hover' feature only works for line object. 
# For point object we need to use 'on_click' on the object then label will be shown below the map.
######
def hover_handler(event=None, id=None, properties=None):
    s = ""
    for key, value in properties.items():
        if str(value)!='None':
            s = s +str(key) +":" + str(value) + ";\t"
    if s=="":
        s = 'No Data'
    label.value = s
    
# Hover lines to show label data.
layer_lines.on_hover(hover_handler)
# Click points to show label data.
layer_points.on_click(hover_handler)
# Add layer to differentiate lines and points. 
map.add_layer(layer_lines)
map.add_layer(layer_points)

# Showing map.
ipyw.VBox([map, label])